### Tokenize and Preprocess

In [1]:
from transformers import BertTokenizer
from transformers import AutoTokenizer ## use AutoTOkenizer will load the fast tokenizer 

- Here’s an example using the BERT tokenizer, which is a WordPiece tokenizer:

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
sequence = "A Titan RTX has 24GB of VRAM"
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)

['A', 'Titan', 'R', '##T', '##X', 'has', '24', '##GB', 'of', 'V', '##RA', '##M']


- each token can be converted to IDs ; we can also decode from ids

In [3]:
inputs = tokenizer(sequence,return_offsets_mapping=True)
encoded_sequence = inputs['input_ids']
print(encoded_sequence)
decoded_sequence = tokenizer.decode(encoded_sequence)
print(decoded_sequence)

[101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102]
[CLS] A Titan RTX has 24GB of VRAM [SEP]


- You can also track see if tokens are came from the same words and also the mapping

In [4]:
print(inputs.tokens())
print(inputs.word_ids())
print(inputs['offset_mapping'])

['[CLS]', 'A', 'Titan', 'R', '##T', '##X', 'has', '24', '##GB', 'of', 'V', '##RA', '##M', '[SEP]']
[None, 0, 1, 2, 2, 2, 3, 4, 4, 5, 6, 6, 6, None]
[(0, 0), (0, 1), (2, 7), (8, 9), (9, 10), (10, 11), (12, 15), (16, 18), (18, 20), (21, 23), (24, 25), (25, 27), (27, 28), (0, 0)]


### Batch Tokenize

- Padding: Tokenizer will automatically pad a batch and provide proper attention masks
- Truncation: On the other end of the spectrum, sometimes a sequence may be too long for a model to handle. In this case, you will need to truncate the sequence to a shorter length. Set the truncation parameter to True to truncate a sequence to the maximum length accepted by the model
- Return Tensor: Set the return_tensors parameter to either pt for PyTorch, or tf for TensorFlow

In [5]:
sequence_a = "This is a short sequence."
sequence_b = "This is a rather long sequence. It is at least longer than the sequence A."
padded_sequences = tokenizer([sequence_a, sequence_b], 
                             padding=True,truncation=True,
                             return_tensors="pt")
print(padded_sequences)

{'input_ids': tensor([[ 101, 1188, 1110,  170, 1603, 4954,  119,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [ 101, 1188, 1110,  170, 1897, 1263, 4954,  119, 1135, 1110, 1120, 1655,
         2039, 1190, 1103, 4954,  138,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### Processing sentence pairs 

- Some models’ purpose is to do classification on pairs of sentences or question answering.These require two different sequences to be joined in a single “input_ids” entry, which usually is performed with the help of special tokens, such as the classifier ([CLS]) and separator ([SEP]) tokens. For example, the BERT model builds its two sequence input as such:

In [6]:
sequence_a = "HuggingFace is based in NYC"
sequence_b = "Where is HuggingFace based?"
encoded_dict = tokenizer(sequence_a, sequence_b)
decoded = tokenizer.decode(encoded_dict["input_ids"])
print(decoded)
print(encoded_dict["token_type_ids"])

[CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Small Tips :
- For models employing the function apply_chunking_to_forward(), the chunk_size defines the number of output embeddings that are computed in parallel and thus defines the trade-off between memory and time complexity. If chunk_size is set to 0, no feed forward chunking is done.
    https://huggingface.co/docs/transformers/glossary#feed-forward-chunking

### Customize Post Processing step 
- say we want to add additional features after the original text 

In [89]:
from tokenizers import processors

In [90]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [91]:
input_text = 'This is just an test sentence. with some posttive words like happy, excited; and some negetive words like however, angry, adverse'
feature1 = 'happy excited'
feature2 = 'however adverse'
input_text2 = '{} [SEP] {}'.format(feature1,feature2)


In [92]:
tokens = tokenizer.tokenize(input_text)
tokenizer.decode(tokenizer.encode(input_text))

'[CLS] This is just an test sentence. with some posttive words like happy, excited ; and some negetive words like however, angry, adverse [SEP]'

In [105]:
cls_token_id = tokenizer.vocab.get('[CLS]')
sep_token_id = tokenizer.vocab.get('[SEP]')

- ### now we can change the post processing step function to formate customized formate
- https://huggingface.co/docs/tokenizers/api/post-processors#tokenizers.processors.TemplateProcessing
- https://huggingface.co/course/chapter6/8?fw=pt

In [98]:
#### bert post processing looks something like this 
'''
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1 [EOS]:2",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)
'''

'\ntokenizer.post_processor = processors.TemplateProcessing(\n    single=f"[CLS]:0 $A:0 [SEP]:0",\n    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1 [EOS]:2",\n    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],\n)\n'

In [108]:
## now let's change it to something different, asusme a senario that we are provideing addtional feature/control as input 2
tokenizer.backend_tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="$A:0 [SEP]:0 $B:1 [SEP]:1 [CLS]:0",  ## we changed the arrange, may not make sense but just as a demo
    special_tokens=[("[SEP]", sep_token_id), ("[CLS]", cls_token_id)],
)

In [109]:
## now we can see the post processing it modified to customized way
res = tokenizer(input_text,input_text2)
print(tokenizer.decode(res['input_ids']))
res

This is just an test sentence. with some posttive words like happy, excited ; and some negetive words like however, angry, adverse [SEP] happy excited [SEP] however adverse [SEP] [CLS]


{'input_ids': [1188, 1110, 1198, 1126, 2774, 5650, 119, 1114, 1199, 2112, 3946, 1734, 1176, 2816, 117, 7215, 132, 1105, 1199, 24928, 16609, 2109, 1734, 1176, 1649, 117, 4259, 117, 16798, 102, 2816, 7215, 102, 1649, 16798, 102, 101], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [110]:
import config
import os
outdir = os.path.join(config.data_folder,'new_tokenizer','customized_post_process')
tokenizer.save_pretrained(outdir)

('../../All_Data/HuggingFace/new_tokenizer/customized_post_process/tokenizer_config.json',
 '../../All_Data/HuggingFace/new_tokenizer/customized_post_process/special_tokens_map.json',
 '../../All_Data/HuggingFace/new_tokenizer/customized_post_process/vocab.txt',
 '../../All_Data/HuggingFace/new_tokenizer/customized_post_process/added_tokens.json',
 '../../All_Data/HuggingFace/new_tokenizer/customized_post_process/tokenizer.json')

In [111]:
## to load tokenizer 
tokenizer = AutoTokenizer.from_pretrained(outdir)
res = tokenizer(input_text,input_text2)
print(tokenizer.decode(res['input_ids'])) ## load it and it is correctly tokenizing inputs 

This is just an test sentence. with some posttive words like happy, excited ; and some negetive words like however, angry, adverse [SEP] happy excited [SEP] however adverse [SEP] [CLS]
